# JEMSSWrapper Basic Test Notebook
This notebook tests loading the JEMSSWrapper and accessing some utilities.

In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
# Ahora intentar cargar
using JEMSSWrapper

  Activating project at `~/Projects/TFG/ExperimentFramework/JEMSSWrapper.jl`
Precompiling packages...
Info Given JEMSSWrapper was explicitly requested, output will be shown live 
[ Info: ✅ Successfully loaded JEMSS from local fork
   3044.2 ms  ✓ JEMSSWrapper
  1 dependency successfully precompiled in 4 seconds. 193 already precompiled.
  1 dependency had output during precompilation:
┌ JEMSSWrapper
│  [Output was shown above]
└  


In [2]:
scenario_name = "auckland"
config_name = "base.toml"
ambulance_file = "ambulances/ambulances_1.csv"
calls_file = "calls/single/train/gen_config.xml"
sim_config_xml_file = joinpath(dirname(@__DIR__), "deps", "JEMSS", "example", "input", "sim_config.xml")

scenario = load_scenario_from_config(scenario_name, config_name, ambulance_file, calls_file);

[ Info: Loading scenario from config: auckland/configs/base.toml
[ Info: Initializing simulation...
[ Info: Scenario loaded successfully!


In [3]:
sim1 = create_simulation_instance(scenario);
sim2 = create_simulation_instance(scenario);

In [4]:
jemss.simulate!(sim1, doPrint=true)

sim time: 421.14    sim duration: 421.14    events simulated: 992475    real duration: 1.79 seconds


true

In [5]:
simulate_custom!(sim2, doPrint=true)

sim time: 421.14    sim duration: 421.14    events simulated: 992475    real duration: 1.37 seconds


true

In [6]:
avg_time = jemss.getAvgCallResponseDuration(sim1, useMinutes=true)
in_time_proportion = jemss.countCallsReachedInTime(sim1) / length(sim1.calls)
println(avg_time)
println(in_time_proportion)

10.614892351182112
0.5686067858842728


In [7]:
avg_time = jemss.getAvgCallResponseDuration(sim2, useMinutes=true)
in_time_proportion = jemss.countCallsReachedInTime(sim2) / length(sim2.calls)
println(avg_time)
println(in_time_proportion)

10.614892351182112
0.5686067858842728


## Define location input features

In [6]:
"""
    Bounds

Estructura para almacenar los límites espaciales de una región rectangular.

# Campos
- `xmin::Float64`: Coordenada x mínima
- `xmax::Float64`: Coordenada x máxima  
- `ymin::Float64`: Coordenada y mínima
- `ymax::Float64`: Coordenada y máxima
"""
struct Bounds
    xmin::Float64
    xmax::Float64
    ymin::Float64
    ymax::Float64

    function Bounds(xmin, xmax, ymin, ymax)
        # Validación de que los límites sean coherentes
        xmin ≤ xmax || throw(ArgumentError("xmin debe ser ≤ xmax"))
        ymin ≤ ymax || throw(ArgumentError("ymin debe ser ≤ ymax"))
        new(xmin, xmax, ymin, ymax)
    end
end

"""
    get_sim_bounds(sim; digits=8)

Calcula los límites espaciales (bounding box) de todos los nodos en la red de simulación.

# Argumentos
- `sim`: Objeto de simulación que contiene la red con nodos
- `digits=8`: Número de dígitos decimales para el redondeo

# Retorna
- `Bounds`: Estructura con los límites xmin, xmax, ymin, ymax

# Ejemplo
```julia
bounds = get_sim_bounds(simulation, digits=6)
println("Área: \$(bounds.xmin) a \$(bounds.xmax) × \$(bounds.ymin) a \$(bounds.ymax)")
```
"""
function get_sim_bounds(sim; digits=8)
    nodes = sim.net.fGraph.nodes
    
    # Verificar que hay nodos
    isempty(nodes) && throw(ArgumentError("No hay nodos en la red"))
    
    # Extraer coordenadas usando broadcasting (más eficiente)
    xs = [n.location.x for n in nodes]
    ys = [n.location.y for n in nodes]

    # Calcular extremos
    xmin, xmax = extrema(xs)
    ymin, ymax = extrema(ys)
    
    # Aplicar redondeo y crear Bounds
    return Bounds(
        round(xmin, digits=digits),
        round(xmax, digits=digits), 
        round(ymin, digits=digits),
        round(ymax, digits=digits)
    )
end

# Constructor de conveniencia alternativo
"""
    Bounds(points::Vector{<:NamedTuple})

Constructor alternativo que acepta un vector de puntos con coordenadas x, y.
"""
function Bounds(points::Vector{T}) where T
    isempty(points) && throw(ArgumentError("Vector de puntos vacío"))
    
    xs = [p.x for p in points]  
    ys = [p.y for p in points]
    
    xmin, xmax = extrema(xs)
    ymin, ymax = extrema(ys)
    
    return Bounds(xmin, xmax, ymin, ymax)
end

Bounds

In [16]:
# ====================================================================
# FUNCIONES PARA OBTENER UBICACIONES
# ====================================================================

"""
    getLocationAmb!(sim, ambulance_id)

Obtiene la ubicación actual de una ambulancia en la simulación.
"""
function getLocationAmb!(sim::jemss.Simulation, ambulance_id::Int)
    @assert 1 ≤ ambulance_id ≤ length(sim.ambulances) "ID de ambulancia inválido: $ambulance_id"
    return jemss.getRouteCurrentLocation!(sim.net, sim.ambulances[ambulance_id].route, sim.time)
end

"""
    getLocationStat(sim, station_id)

Obtiene la ubicación de una estación en la simulación.
"""
function getLocationStat(sim::jemss.Simulation, station_id::Int)
    @assert 1 ≤ station_id ≤ length(sim.stations) "ID de estación inválido: $station_id"
    return sim.stations[station_id].location
end

# ====================================================================
# FUNCIONES PARA CONVERTIR UBICACIONES A MATRICES
# ====================================================================

"""
    locations_to_matrix(locs)

Convierte un vector de ubicaciones a una matriz Nx2 (x, y).
"""
function locations_to_matrix(locs::Vector{jemss.Location})
    n = length(locs)
    n == 0 && return Matrix{Float64}(undef, 0, 2)
    
    mat = Matrix{Float64}(undef, n, 2)
    @inbounds for i in 1:n
        mat[i, 1] = locs[i].x
        mat[i, 2] = locs[i].y
    end
    return mat
end

# ====================================================================
# NORMALIZACIÓN MIN-MAX
# ====================================================================

"""
    minmax_scale(mat, bounds; target_range=(-1.0, 1.0))

Aplica normalización min-max a una matriz de coordenadas usando bounds específicos.

# Argumentos
- `mat::Matrix{Float64}`: Matriz Nx2 con coordenadas (x, y)
- `bounds::Bounds`: Objeto con límites xmin, xmax, ymin, ymax
- `target_range`: Tupla con el rango objetivo (por defecto (-1, 1))

# Retorna
- `Matrix{Float64}`: Matriz normalizada al rango especificado
"""
function minmax_scale(mat::Matrix{Float64}, bounds::Bounds; target_range=(-1.0, 1.0))
    size(mat, 2) != 2 && throw(ArgumentError("La matriz debe tener exactamente 2 columnas (x, y)"))
    
    if size(mat, 1) == 0
        return mat  # Retorna matriz vacía si no hay datos
    end
    
    # Extraer rangos objetivo
    target_min, target_max = target_range
    target_span = target_max - target_min
    
    # Crear matriz de salida
    scaled_mat = similar(mat)
    
    # Normalizar coordenadas X (columna 1)
    x_span = bounds.xmax - bounds.xmin
    if x_span > 0
        @inbounds for i in 1:size(mat, 1)
            # Normalizar a [0, 1] y luego escalar al rango objetivo
            normalized_x = (mat[i, 1] - bounds.xmin) / x_span
            scaled_mat[i, 1] = target_min + normalized_x * target_span
        end
    else
        # Si no hay variación en X, asignar el punto medio del rango
        scaled_mat[:, 1] .= (target_min + target_max) / 2
    end
    
    # Normalizar coordenadas Y (columna 2)  
    y_span = bounds.ymax - bounds.ymin
    if y_span > 0
        @inbounds for i in 1:size(mat, 1)
            # Normalizar a [0, 1] y luego escalar al rango objetivo
            normalized_y = (mat[i, 2] - bounds.ymin) / y_span
            scaled_mat[i, 2] = target_min + normalized_y * target_span
        end
    else
        # Si no hay variación en Y, asignar el punto medio del rango
        scaled_mat[:, 2] .= (target_min + target_max) / 2
    end
    
    return scaled_mat
end

# ====================================================================
# FUNCIONES DE ALTO NIVEL PARA EXTRAER COORDENADAS
# ====================================================================

"""
    get_ambulance_coordinates(sim; normalize=true, bounds=nothing, target_range=(-1.0, 1.0))

Extrae las coordenadas de todas las ambulancias en la simulación.

# Argumentos
- `sim`: Objeto de simulación
- `normalize=true`: Si aplicar normalización min-max
- `bounds=nothing`: Bounds para normalización (se calculan automáticamente si es nothing)
- `target_range=(-1.0, 1.0)`: Rango objetivo para normalización

# Retorna
- `Matrix{Float64}`: Matriz Nx2 con coordenadas de ambulancias
"""
function get_ambulance_coordinates(sim::jemss.Simulation; 
                                 normalize=true, 
                                 bounds=nothing, 
                                 target_range=(-1.0, 1.0))
    # Obtener ubicaciones de todas las ambulancias
    ambulance_locs = [getLocationAmb!(sim, i) for i in 1:length(sim.ambulances)]
    
    # Convertir a matriz
    coord_matrix = locations_to_matrix(ambulance_locs)
    
    # Aplicar normalización si se solicita
    if normalize && size(coord_matrix, 1) > 0
        if bounds === nothing
            bounds = get_sim_bounds(sim)
        end
        coord_matrix = minmax_scale(coord_matrix, bounds; target_range=target_range)
    end
    
    return coord_matrix
end

"""
    get_station_coordinates(sim; normalize=true, bounds=nothing, target_range=(-1.0, 1.0))

Extrae las coordenadas de todas las estaciones en la simulación.

# Argumentos
- `sim`: Objeto de simulación  
- `normalize=true`: Si aplicar normalización min-max
- `bounds=nothing`: Bounds para normalización (se calculan automáticamente si es nothing)
- `target_range=(-1.0, 1.0)`: Rango objetivo para normalización

# Retorna
- `Matrix{Float64}`: Matriz Nx2 con coordenadas de estaciones
"""
function get_station_coordinates(sim::jemss.Simulation; 
                               normalize=true, 
                               bounds=nothing, 
                               target_range=(-1.0, 1.0))
    # Obtener ubicaciones de todas las estaciones
    station_locs = [getLocationStat(sim, i) for i in 1:length(sim.stations)]
    
    # Convertir a matriz
    coord_matrix = locations_to_matrix(station_locs)
    
    # Aplicar normalización si se solicita
    if normalize && size(coord_matrix, 1) > 0
        if bounds === nothing
            bounds = get_sim_bounds(sim)
        end
        coord_matrix = minmax_scale(coord_matrix, bounds; target_range=target_range)
    end
    
    return coord_matrix
end

# ====================================================================
# EJEMPLO DE USO
# ====================================================================

"""
# Ejemplos de uso:

# 1. Obtener solo coordenadas de ambulancias normalizadas
ambulance_coords = get_ambulance_coordinates(sim)

# 2. Obtener coordenadas sin normalizar
raw_coords = get_ambulance_coordinates(sim, normalize=false)

# 3. Normalizar a un rango diferente [0, 1]
coords_01 = get_ambulance_coordinates(sim, target_range=(0.0, 1.0))
"""

"# Ejemplos de uso:\n\n# 1. Obtener solo coordenadas de ambulancias normalizadas\nambulance_coords = get_ambulance_coordinates(sim)\n\n# 2. Obtener coordenadas sin normalizar\nraw_coords = get_ambulance_coordinates(sim, normalize=false)\n\n# 3. Normalizar a un rango diferente [0, 1]\ncoords_01 = get_ambulance_coordinates(sim, target_range=(0.0, 1.0))\n"

In [27]:
"""
    MyMoveUpData

Estructura para almacenar información estática de la simulación que se reutiliza
en múltiples momentos temporales.

# Campos
- `bounds::Bounds`: Límites espaciales de la simulación
- `target_range::Tuple{Float64, Float64}`: Rango objetivo para normalización (ej. (-1.0, 1.0))
- `station_coordinates::Matrix{Float64}`: Coordenadas normalizadas de estaciones (Mx2)

# Ejemplo
```julia
data = MyMoveUpData(sim)
# Usar en diferentes momentos:
coords1 = get_complete_coordinates(sim, data, time=100.0)
coords2 = get_complete_coordinates(sim, data, time=200.0)
```
"""
struct MyMoveUpData
    bounds::Bounds
    target_range::Tuple{Float64, Float64}
    station_coordinates::Matrix{Float64}
    
    # Constructor interno para validaciones
    function MyMoveUpData(bounds::Bounds, target_range::Tuple{Float64, Float64}, station_coordinates::Matrix{Float64})
        # Validar que target_range sea válido
        target_range[1] < target_range[2] || throw(ArgumentError("target_range debe ser (min, max) con min < max"))
        
        # Validar que station_coordinates tenga 2 columnas
        size(station_coordinates, 2) == 2 || throw(ArgumentError("station_coordinates debe tener exactamente 2 columnas (x, y)"))
        
        new(bounds, target_range, station_coordinates)
    end
end

# ====================================================================
# CONSTRUCTORES DE CONVENIENCIA
# ====================================================================

"""
    MyMoveUpData(sim; target_range=(-1.0, 1.0))

Constructor principal que extrae toda la información estática de la simulación.

# Argumentos
- `sim::jemss.Simulation`: Objeto de simulación
- `target_range=(-1.0, 1.0)`: Rango objetivo para normalización

# Retorna
- `MyMoveUpData`: Estructura con datos estáticos precalculados
"""
function MyMoveUpData(sim::jemss.Simulation; target_range=(-1.0, 1.0))
    # Calcular bounds de toda la simulación
    bounds = get_sim_bounds(sim)
    
    # Obtener y normalizar coordenadas de estaciones (estas no cambian)
    station_coords = get_station_coordinates(sim; normalize=true, bounds=bounds, target_range=target_range)
    
    return MyMoveUpData(bounds, target_range, station_coords)
end

# ====================================================================
# FUNCIONES PARA USAR CON MyMoveUpData
# ====================================================================

"""
    get_current_ambulance_coordinates(sim, data::MyMoveUpData)

Obtiene las coordenadas normalizadas de ambulancias en el momento actual usando
los parámetros almacenados en `data`.

# Argumentos
- `sim::jemss.Simulation`: Objeto de simulación
- `data::MyMoveUpData`: Datos estáticos precalculados

# Retorna
- `Matrix{Float64}`: Coordenadas normalizadas de ambulancias (Nx2)
"""
function get_current_ambulance_coordinates(sim::jemss.Simulation, data::MyMoveUpData)
    return get_ambulance_coordinates(sim; 
                                   normalize=true, 
                                   bounds=data.bounds, 
                                   target_range=data.target_range)
end

"""
    get_complete_coordinates(sim, data::MyMoveUpData)

Combina las coordenadas actuales de ambulancias con las estaciones precalculadas.

# Argumentos
- `sim::jemss.Simulation`: Objeto de simulación
- `data::MyMoveUpData`: Datos estáticos precalculados

# Retorna
- `combined_coords::Array{Float64, 2}`: Matriz con las coordenadas de las ambulancias y estaciones
"""
function get_complete_coordinates(sim::jemss.Simulation, data::MyMoveUpData)
    # Obtener coordenadas actuales de ambulancias
    ambulance_coords = get_current_ambulance_coordinates(sim, data)
    
    # Combinar con estaciones precalculadas
    combined_coords = vcat(ambulance_coords, data.station_coordinates)
    
    return combined_coords
end

# ====================================================================
# EJEMPLO DE USO COMPLETO
# ====================================================================

"""
# Flujo de trabajo típico:

# 1. Inicialización (una sola vez al inicio)
data = MyMoveUpData(sim, target_range=(-1.0, 1.0))

# 2. Durante la simulación (múltiples veces)
# En diferentes momentos temporales:
sim.time = 100.0
coords_t100 = get_complete_coordinates(sim, data)

sim.time = 200.0  
coords_t200 = get_complete_coordinates(sim, data)
"""

"# Flujo de trabajo típico:\n\n# 1. Inicialización (una sola vez al inicio)\ndata = MyMoveUpData(sim, target_range=(-1.0, 1.0))\n\n# 2. Durante la simulación (múltiples veces)\n# En diferentes momentos temporales:\nsim.time = 100.0\ncoords_t100 = get_complete_coordinates(sim, data)\n\nsim.time = 200.0  \ncoords_t200 = get_complete_coordinates(sim, data)\n\n# 3. Acceso a información\nprintln(\"Número de estaciones: \", get_station_count(data))\nprintln(\"Bounds: \", get_bounds(data))\nprintln(\"Ambulancias en t=100: \", size(coords_t100.ambulances, 1))\n"

In [51]:
# ====================================================================
# TRANSFORMACIÓN DE COORDENADAS CENTRADAS EN AMBULANCIA
# ====================================================================

"""
    center_on_ambulance(coords::Matrix{Float64}, ambulance_index::Int)

Transforma las coordenadas para usar una ambulancia específica como origen del sistema 
de coordenadas. Todas las coordenadas se expresan relativas a la ambulancia seleccionada.

# Argumentos
- `coords::Matrix{Float64}`: Matriz de coordenadas (N×2) donde las primeras filas son ambulancias
- `ambulance_index::Int`: Índice de la ambulancia que será el nuevo origen (1-based)

# Retorna
- `Matrix{Float64}`: Matriz transformada donde la ambulancia seleccionada está en (0,0)

# Ejemplo
```julia
coords = get_complete_coordinates(sim, data)
centered_coords = center_on_ambulance(coords, 1)  # Primera ambulancia como origen
```
"""
function center_on_ambulance(coords::Matrix{Float64}, ambulance_index::Int)
    n_points = size(coords, 1)
    
    # Validaciones
    n_points == 0 && throw(ArgumentError("Matriz de coordenadas vacía"))
    size(coords, 2) != 2 && throw(ArgumentError("La matriz debe tener exactamente 2 columnas (x, y)"))
    1 ≤ ambulance_index ≤ n_points || throw(BoundsError("ambulance_index debe estar entre 1 y $(n_points)"))
    
    # Obtener coordenadas de la ambulancia de referencia
    origin_x = coords[ambulance_index, 1]
    origin_y = coords[ambulance_index, 2]
    
    # Crear matriz centrada
    centered_coords = similar(coords)
    @inbounds for i in 1:n_points
        centered_coords[i, 1] = coords[i, 1] - origin_x
        centered_coords[i, 2] = coords[i, 2] - origin_y
    end
    
    return centered_coords
end

"""
    get_ambulance_centered_coordinates(sim::jemss.Simulation, data::MyMoveUpData, ambulance_index::Int)

Función de conveniencia que combina la obtención de coordenadas completas y el centrado
en una ambulancia específica.

# Argumentos
- `sim::jemss.Simulation`: Objeto de simulación
- `data::MyMoveUpData`: Datos estáticos precalculados  
- `ambulance_index::Int`: Índice de la ambulancia que será el origen (1-based)

# Retorna
- `Matrix{Float64}`: Matriz de coordenadas centradas en la ambulancia seleccionada

# Ejemplo
```julia
data = MyMoveUpData(sim)
centered_coords = get_ambulance_centered_coordinates(sim, data, 1)
```
"""
function get_ambulance_centered_coordinates(sim::jemss.Simulation, data::MyMoveUpData, ambulance_index::Int)
    # Obtener coordenadas completas
    coords = get_complete_coordinates(sim, data)
    
    # Centrar en la ambulancia especificada
    return center_on_ambulance(coords, ambulance_index)
end

"""
# Ejemplo de flujo completo:

# 1. Preparar datos
data = MyMoveUpData(sim)

# 2. En cada time step:
sim.time = 100.0

# Opción A: Dos pasos
coords = get_complete_coordinates(sim, data)
centered_coords = center_on_ambulance(coords, 1)  # Primera ambulancia como origen

# Opción B: Un solo paso  
centered_coords = get_ambulance_centered_coordinates(sim, data, 1)

# 3. Usar coordenadas centradas para ML
ml_input = centered_coords  # Matriz lista para algoritmos
"""

get_ambulance_centered_coordinates

In [28]:
data = MyMoveUpData(sim, target_range=(-1.0, 1.0))

MyMoveUpData:
  Bounds: (174.5615016, -37.1078986) to (175.00306285, -36.5243147)
  Target range: (-1.0, 1.0)
  Stations: 14 locations


In [56]:
get_ambulance_centered_coordinates(sim, data, 7)

30×2 Matrix{Float64}:
  0.623053    -0.558045
  0.691333    -0.396101
 -0.289178    -0.434693
 -0.289178    -0.434693
  0.506702    -0.744273
  0.506702    -0.744273
  0.0          0.0
  0.00563455  -0.251385
  0.00563455  -0.251385
 -0.51023     -0.283466
 -0.51023     -0.283466
  0.457699    -0.269517
 -0.136153     0.159963
  ⋮           
 -0.289178    -0.434693
  0.506702    -0.744273
  0.812897    -0.971511
  0.0          0.0
  0.00563455  -0.251385
 -0.366074     0.569903
 -0.51023     -0.283466
  0.457699    -0.269517
 -0.136153     0.159963
  0.385111    -0.529274
  0.313542    -0.394963
 -0.0176284   -0.44613

In [37]:
all_coordinates

30×2 Matrix{Float64}:
  0.510306  -0.444342
  0.578587  -0.282398
 -0.401925  -0.32099
 -0.401925  -0.32099
  0.393956  -0.63057
  0.393956  -0.63057
 -0.112746   0.113703
 -0.107112  -0.137681
 -0.107112  -0.137681
 -0.622977  -0.169763
 -0.622977  -0.169763
  0.344952  -0.155814
 -0.2489     0.273666
  ⋮         
 -0.401925  -0.32099
  0.393956  -0.63057
  0.700151  -0.857808
 -0.112746   0.113703
 -0.107112  -0.137681
 -0.47882    0.683606
 -0.622977  -0.169763
  0.344952  -0.155814
 -0.2489     0.273666
  0.272364  -0.415571
  0.200796  -0.28126
 -0.130375  -0.332426

In [45]:
all_coordinates

30×2 Matrix{Float64}:
  0.510306  -0.444342
  0.578587  -0.282398
 -0.401925  -0.32099
 -0.401925  -0.32099
  0.393956  -0.63057
  0.393956  -0.63057
 -0.112746   0.113703
 -0.107112  -0.137681
 -0.107112  -0.137681
 -0.622977  -0.169763
 -0.622977  -0.169763
  0.344952  -0.155814
 -0.2489     0.273666
  ⋮         
 -0.401925  -0.32099
  0.393956  -0.63057
  0.700151  -0.857808
 -0.112746   0.113703
 -0.107112  -0.137681
 -0.47882    0.683606
 -0.622977  -0.169763
  0.344952  -0.155814
 -0.2489     0.273666
  0.272364  -0.415571
  0.200796  -0.28126
 -0.130375  -0.332426

In [43]:
amb_id = 5
amb_x, amb_y = all_coordinates[amb_id, :]
all_coordinates[:, 1] .- amb_x
all_coordinates[:, 2] .- amb_y

30-element Vector{Float64}:
  0.18622857827296546
  0.34817273060479736
  0.30958016490859663
  0.30958016490859663
  0.0
  0.0
  0.7442734455148442
  0.4928888545417218
  0.4928888545417218
  0.4608077775963215
  0.4608077775963215
  0.4747560719204065
  0.904236734426704
  ⋮
  0.30958016490859663
  0.0
 -0.22723724900568476
  0.7442734455148442
  0.4928888545417218
  1.3141760764818842
  0.4608077775963215
  0.4747560719204065
  0.904236734426704
  0.21499907725348955
  0.3493105275865175
  0.2981439344025737

In [15]:
struct MyMoveUpData
    bounds::Bounds
    target_range::Tuple{Float64, Float64}
    station_coordinates::Array{Float64, 2}
end

In [14]:
typeof(get_station_coordinates(sim))

Matrix{Float64} (alias for Array{Float64, 2})

In [8]:
get_ambulance_coordinates(sim)

16×2 Matrix{Float64}:
  0.510306  -0.444342
  0.578587  -0.282398
 -0.401925  -0.32099
 -0.401925  -0.32099
  0.393956  -0.63057
  0.393956  -0.63057
 -0.112746   0.113703
 -0.107112  -0.137681
 -0.107112  -0.137681
 -0.622977  -0.169763
 -0.622977  -0.169763
  0.344952  -0.155814
 -0.2489     0.273666
 -0.2489     0.273666
  0.272364  -0.415571
 -0.130375  -0.332426

In [135]:
struct Bounds
    xmin::Float64
    xmax::Float64
    ymin::Float64
    ymax::Float64

    Bound(xmin, xmax, ymin, ymax) = new(xmin, xmax, ymin, ymax)
end
    
function get_sim_bounds(sim, digits=8)
    nodes = sim.net.fGraph.nodes
    xs = [n.location.x for n in nodes]
    ys = [n.location.y for n in nodes]

    xmin, xmax = extrema(xs)
    ymin, ymax = extrema(ys)
    
    return Bounds(map(z -> round(z, digits=digits), [xmin, xmax, ymin, ymax]))
end


getLocationAmb!(sim::jemss.Simulation, ambulance_id::Int) = jemss.getRouteCurrentLocation!(sim.net, sim.ambulances[ambulance_id].route, sim.time)
getLocationStat(sim::jemss.Simulation, station_id::Int) = sim.stations[station_id].location

function locations_to_matrix(locs::Vector{jemss.Location})
    n = length(locs)
    mat = Matrix{Float64}(undef, n, 2)  # prealocamos Nx2
    @inbounds for i in 1:n
        mat[i, 1] = locs[i].x
        mat[i, 2] = locs[i].y
    end
    return mat
end

function minmax_scale(mat::Matrix{Float64}, bounds, range=(-1.0, 1.0))
    a, b = range
    mins = mapslices(minimum, mat; dims=1)  # min por columna
    maxs = mapslices(maximum, mat; dims=1)  # max por columna

    # Broadcasting columna a columna
    scaled = (b - a) .* (mat .- mins) ./ (maxs .- mins) .+ a
    return scaled
end

LoadError: invalid redefinition of type Bounds

In [136]:
get_sim_bounds(sim)

LoadError: MethodError: no method matching Bounds(::Vector{Float64})

In [123]:
bounds = bounds(sim)
location_ambulances = [getLocationAmb!(sim, i) for i in 1:sim.numAmbs]
minmax_scale(locations_to_matrix(location_ambulances))

16×2 Matrix{Float64}:
  0.886347  -0.588098
  1.0       -0.229908
 -0.632059  -0.315267
 -0.632059  -0.315267
  0.692682  -1.0
  0.692682  -1.0
 -0.150723   0.646192
 -0.141344   0.0901766
 -0.141344   0.0901766
 -1.0        0.0192193
 -1.0        0.0192193
  0.611116   0.0500703
 -0.377349   1.0
 -0.377349   1.0
  0.490293  -0.524463
 -0.180065  -0.340562

In [129]:
xmax

175.00306285

In [124]:
location_ambulances

16-element Vector{JEMSS.Location}:
 JEMSS.Location(174.894948, -36.945762)
 JEMSS.Location(174.910023, -36.898508)
 JEMSS.Location(174.693545, -36.909769)
 JEMSS.Location(174.693545, -36.909769)
 JEMSS.Location(174.86926, -37.000102)
 JEMSS.Location(174.86926, -37.000102)
 JEMSS.Location(174.75739, -36.782929)
 JEMSS.Location(174.758634, -36.856281)
 JEMSS.Location(174.758634, -36.856281)
 JEMSS.Location(174.644741, -36.865642)
 JEMSS.Location(174.644741, -36.865642)
 JEMSS.Location(174.858441, -36.861572)
 JEMSS.Location(174.72733, -36.736253)
 JEMSS.Location(174.72733, -36.736253)
 JEMSS.Location(174.842415, -36.937367)
 JEMSS.Location(174.753498, -36.913106)